# Deuxième essai BM25 pour requêter les DPEFs et chercher des sujets spécifiques.

In [60]:
%load_ext autoreload
%autoreload 2

# general libraries
import pandas as pd; pd.set_option('display.max_colwidth', -1)
import numpy as np

# NLP libraries
from spacy.lang.fr import French
import spacy
# from spacy.cli.download import download
# download('fr_core_news_md')

# custom libraries
import scoring

# indexing
# Microsoft Visual C++ 14.0 is required. 
# Get it with "Build Tools for Visual Studio": https://visualstudio.microsoft.com/downloads/
# CGA: can't install right now...
# !pip install annoy
# from annoy import AnnoyIndex

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# DATA IMPORT
dpef_entreprises = "../../data/processed/DPEFs/dpef_paragraphs_sentences.csv"
df = pd.read_csv(dpef_entreprises, sep=";")
print(df.shape)
df = df.loc[0:1000]
documents = df["sentence"].values.tolist() # strings

(23874, 9)


In [34]:
nlp = spacy.load('fr_core_news_sm')

In [35]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x195569530c8>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x1955cfe1c48>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x1955cfe1d68>)]

In [20]:
# instantiate the bm25 scorer using only words and not punctuation # <1min for 19k sentences
documents_words = []
for doc in nlp.pipe(documents, disable=["parser","ner"]):
    documents_words.append([token.text for token in doc if token.pos_!="PUNCT"])
bm25 = scoring.Scoring.create("bm25")

In [21]:
bm25.index(documents_words)

In [24]:
import pickle
with open("../../data/model/words_scorer.pckl", "wb") as f:
    pickle.dump(bm25, f)

In [25]:
bm25_loaded = pickle.load(open("../../data/model/words_scorer.pckl", "rb"))

In [33]:
[token.pos_ for token in nlp("Ceci est un test pollution marine")]

['PRON', 'AUX', 'DET', 'NOUN', 'NOUN', 'ADJ']

In [28]:
bm25_loaded.weights("Ceci est un test pollution marine".split())

[6.477348923759632,
 2.0559555321585474,
 1.8257452855735985,
 6.477348923759632,
 6.85257517701197,
 6.477348923759632]

In [29]:
nlp = spacy.load('fr_core_news_md') # long because of vectors

In [43]:
# Add custom calculation of sentence weight
# Improvement: removal of rare words, etc.
# Improvement: add LSA model to reduce noise in embeddings...
def overwrite_vector_with_weights(doc, scorer=bm25):
    " Score each non-punctuation token from spacy doc, and overwrite the vector representation using BM25 weights"
    word_tokens = [token.text for token in doc if token.pos_ != "PUNCT"]
    word_vectors = [token.vector for token in doc if token.pos_ != "PUNCT"]
    weights = bm25.weights(word_tokens)
    doc.vector = np.average(word_vectors, weights=np.array(weights, dtype=np.float32),axis=0)
    return doc
if "overwrite_vector_with_weights" in nlp.pipeline:
    nlp.replace_pipe("overwrite_vector_with_weights", overwrite_vector_with_weights)
else:
    nlp.add_pipe(overwrite_vector_with_weights)

In [44]:
# transform all documents and overwrite their vector using BM25 weights
documents = list(nlp.pipe(documents)) # else only done once

In [65]:
new_doc = nlp('préservation biodiversité marine')
N_TO_RETRIEVE = 10
similarities = np.array([new_doc.similarity(doc) for doc in documents])
best_matches = np.argsort(-similarities)[:10] # top n
best_matches, similarities[best_matches]

(array([14578, 14931,  4271,  2955, 14928,   910,  4272, 16197, 13833,
         2976], dtype=int64),
 array([0.83305887, 0.75814401, 0.7455708 , 0.69594272, 0.69349737,
        0.64909942, 0.64175053, 0.63775241, 0.63639142, 0.63110251]))

In [66]:
for bet_match_i in best_matches:
    print(documents[bet_match_i], similarities[bet_match_i])

● le label Marine Stewardship Council (MSC) pour la pêche responsable, qui garantit non-pratique de la surpêche et le respect de l’environnement marin ; 0.8330588671578245
Moderniser la flotte pour la rendre plus propre et silencieuse Carrefour modernise sa flotte de véhicules. 0.7581440102112053
Eiffage met en œuvre en 2018 et 2019 des dispositifs biogènes et à biodiversité positive en milieu marin dans le cadre des travaux d’agran- dissement de Port-Haliguen (Morbihan), expérimentation soutenue par l’Agence française pour la biodiversité (AFB) dans le cadre du PIA 2 biodiversité. 0.7455707953751177
sur la biodiversité dans les infrastructures (lutte contre les espèces invasives, mesures compensatoires, accueil de la biodiversité marine, etc.) et la biodiversité urbaine (toitures végétalisées, démarche en développement 0.6959427199222613
● moderniser la flotte pour la rendre plus propre et silencieuse ; 0.6934973688803485
dans différents environnements, avec l’approfondissement des ét

In [4]:
a = range(10)
for x in a:
    pass 
a[9]

9

In [68]:
best_df = df.loc[best_matches]
best_df["score"] = similarities[best_matches]
best_df[["denomination", "sentence", "paragraph", "score"]]

,denomination,sentence,paragraph,score
14578,Carrefour,"● le label Marine Stewardship Council (MSC) pour la pêche responsable, qui garantit non-pratique de la surpêche et le respect de l’environnement marin ;","● le label Marine Stewardship Council (MSC) pour la pêche responsable, qui garantit non-pratique de la surpêche et le respect de l’environnement marin ;",0.833059
14931,Carrefour,Moderniser la flotte pour la rendre plus propre et silencieuse Carrefour modernise sa flotte de véhicules.,"Moderniser la flotte pour la rendre plus propre et silencieuse Carrefour modernise sa flotte de véhicules. Plus de 200 camions roulent désormais au biométhane carburant, moins polluants et moins bruyants. La flotte Carrefour compte plus de 200 camions certifiés selon la norme PIEK Carrefour et permettent aux magasins d’être certifiés Certibruit, moins de 60 dB. Fin 2018, une vingtaine de magasins de Carrefour France situés dans les zones urbaines denses sont labellisés Certibruit®. Par ailleurs, le Groupe s’engage et lutte contre livraisons de nuit. Avec ses partenaires du Cercle des transporteurs, Carrefour France a développé une flotte de véhicules roulant au biométhane. Les bénéfices immédiats du biométhane par rapport à un camion de même capacité diesel Euro 6 sont la diminution de 75 % des émissions de CO2, la suppression des émissions de particules fines et",0.758144
4271,Eiffage,"Eiffage met en œuvre en 2018 et 2019 des dispositifs biogènes et à biodiversité positive en milieu marin dans le cadre des travaux d’agran- dissement de Port-Haliguen (Morbihan), expérimentation soutenue par l’Agence française pour la biodiversité (AFB) dans le cadre du PIA 2 biodiversité.","Eiffage met en œuvre en 2018 et 2019 des dispositifs biogènes et à biodiversité positive en milieu marin dans le cadre des travaux d’agran- dissement de Port-Haliguen (Morbihan), expérimentation soutenue par l’Agence française pour la biodiversité (AFB) dans le cadre du PIA 2 biodiversité. Investi dans ce domaine depuis 2013 via des partenariats avec des ingénieries innovantes (Seaboost, Créocéan, etc.), Eiffage cible d’autres marchés pour ces actions de génie écologique marin, à l’image de l’éolien offshore.",0.745571
2955,Bouygues,"sur la biodiversité dans les infrastructures (lutte contre les espèces invasives, mesures compensatoires, accueil de la biodiversité marine, etc.) et la biodiversité urbaine (toitures végétalisées, démarche en développement","sur la biodiversité dans les infrastructures (lutte contre les espèces invasives, mesures compensatoires, accueil de la biodiversité marine, etc.) et la biodiversité urbaine (toitures végétalisées, démarche en développement",0.695943
14928,Carrefour,● moderniser la flotte pour la rendre plus propre et silencieuse ;,● moderniser la flotte pour la rendre plus propre et silencieuse ;,0.693497
910,Michelin,"dans différents environnements, avec l’approfondissement des études toxicologiques et éco toxicologiques sur le vieillissement à long terme des TRWP ainsi que leur éventuelle toxicité en milieu marin.","dans différents environnements, avec l’approfondissement des études toxicologiques et éco toxicologiques sur le vieillissement à long terme des TRWP ainsi que leur éventuelle toxicité en milieu marin.",0.649099
4272,Eiffage,"Investi dans ce domaine depuis 2013 via des partenariats avec des ingénieries innovantes (Seaboost, Créocéan, etc.), Eiffage cible d’autres marchés pour ces actions de génie écologique marin, à l’image de l’éolien offshore.","Eiffage met en œuvre en 2018 et 2019 des dispositifs biogènes et à biodiversité positive en milieu marin dans le cadre des travaux d’agran- dissement de Port-Haliguen (Morbihan), expérimentation soutenue par l’Agence française pour la biodiversité (AFB) dans le cadre du PIA 2 biodiversité. Investi dans ce domaine depuis 2013 via des partenariats avec des ingénieries innovantes (Seaboost, Créocéan, etc.), Eiffage cible d’autres marchés pour ces actions de génie écologique marin,